In [60]:
import fastf1
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error
# Enable FastF1 caching
fastf1.Cache.enable_cache("f1_cache")

In [61]:
def load_all_laps(year=2024, session_type="R"):
    all_laps = []
    for rnd in range(1, 21):  # Try 20 races for 2025 (adjust as needed)
        try:
            session = fastf1.get_session(year, rnd, session_type)
            session.load()
            laps = session.laps[["Driver", "LapTime"]].dropna()
            laps["LapTime (s)"] = laps["LapTime"].dt.total_seconds()
            all_laps.append(laps)
            print(f"✅ Loaded race {rnd}")
        except Exception as e:
            print(f"❌ Could not load race {rnd}: {e}")
    
    if not all_laps:
        print("❌ No race data loaded.")
        return pd.DataFrame(columns=["Driver", "LapTime (s)"])
    
    return pd.concat(all_laps, ignore_index=True)

In [62]:
def predict_winner():
    # Step 1: Load all 2025 race lap times
    laps_2025 = load_all_laps(year=2024)  # Ensure this function loads 2025 data

    # Step 2: Average lap time per driver (excluding missing values)
    laps_2025.dropna(subset=["Driver", "LapTime (s)"], inplace=True)
    avg_laps = laps_2025.groupby("Driver")["LapTime (s)"].mean().reset_index()

    # Step 3: 2025 Chinese GP Qualifying Times (Retain original order)
    qualifying_2025 = pd.DataFrame({
        "Driver": ["Oscar Piastri", "George Russell", "Lando Norris", "Max Verstappen", "Lewis Hamilton",
                   "Charles Leclerc", "Isack Hadjar", "Andrea Kimi Antonelli", "Yuki Tsunoda", "Alexander Albon",
                   "Esteban Ocon", "Nico Hülkenberg", "Fernando Alonso", "Lance Stroll", "Carlos Sainz Jr.",
                   "Pierre Gasly", "Oliver Bearman", "Jack Doohan", "Gabriel Bortoleto", "Liam Lawson"],
        "QualifyingTime (s)": [90.641, 90.723, 90.793, 90.817, 90.927,
                               91.021, 91.079, 91.103, 91.638, 91.706,
                               91.625, 91.632, 91.688, 91.773, 91.840,
                               91.992, 92.018, 92.092, 92.141, 92.174]
    })

    driver_mapping = {
        "Oscar Piastri": "PIA", "George Russell": "RUS", "Lando Norris": "NOR", "Max Verstappen": "VER",
        "Lewis Hamilton": "HAM", "Charles Leclerc": "LEC", "Isack Hadjar": "HAD", "Andrea Kimi Antonelli": "ANT",
        "Yuki Tsunoda": "TSU", "Alexander Albon": "ALB", "Esteban Ocon": "OCO", "Nico Hülkenberg": "HUL",
        "Fernando Alonso": "ALO", "Lance Stroll": "STR", "Carlos Sainz Jr.": "SAI", "Pierre Gasly": "GAS",
        "Oliver Bearman": "BEA", "Jack Doohan": "DOO", "Gabriel Bortoleto": "BOR", "Liam Lawson": "LAW"
    }
    
    qualifying_2025["DriverCode"] = qualifying_2025["Driver"].map(driver_mapping)

    # Step 4: Merge with training data (matching driver codes)
    merged_data = qualifying_2025.merge(avg_laps, left_on="DriverCode", right_on="Driver", how="inner")

    # Step 5: Model training with scaled data
    X = merged_data[["QualifyingTime (s)"]]
    y = merged_data["LapTime (s)"]

    # Scaling the feature
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=39)
    model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=39)
    model.fit(X_train, y_train)

    # Step 6: Predict all 2025 drivers (also scaled)
    X_qualifying_scaled = scaler.transform(qualifying_2025[["QualifyingTime (s)"]])
    predictions = model.predict(X_qualifying_scaled)
    qualifying_2025["PredictedRaceTime (s)"] = predictions

    # Retain original order (qualifying order)
    qualifying_2025["StartingOrder"] = range(0, len(qualifying_2025))
    qualifying_2025 = qualifying_2025.sort_values(by="StartingOrder").reset_index(drop=True)
    
    # Display the results
    print("\n🏁 Predicted 2025 Chinese GP Winner (Full Season Trained) 🏁\n")
    print(qualifying_2025[["StartingOrder", "Driver", "PredictedRaceTime (s)"]])

    # Model Performance
    mae = mean_absolute_error(y_test, model.predict(X_test))
    print(f"\n🔍 MAE on test split: {mae:.2f} seconds")

    winner = qualifying_2025.iloc[0]["Driver"]
    return winner

In [63]:
#oad_all_laps(2024,"R")
predict_winner()

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
core           INFO 	Loading data for Saudi Arabian Grand Prix

✅ Loaded race 1


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '16', '81', '14', '63', '38', '4', '44', '27', '23', '20', '31', '2', '22', '3', '77', '24', '18', '10']
core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


✅ Loaded race 2


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 19 drivers: ['55', '16', '4', '81', '11', '18', '22', '14', '27', '20', '23', '3', '10', '77', '24', '31', '63', '44', '1']
core           INFO 	Loading data for Japanese Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


...

req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '55', '1', '4', '81', '63', '11', '27', '30', '43', '20', '10', '14', '22', '18', '23', '77', '31', '24', '44']
core           INFO 	Loading data for Mexico City Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


✅ Loaded race 19


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['55', '4', '16', '44', '63', '1', '20', '81', '27', '10', '18', '43', '31', '77', '24', '30', '11', '14', '23', '22']


✅ Loaded race 20

🏁 Predicted 2025 Chinese GP Winner (Full Season Trained) 🏁

    StartingOrder                 Driver  PredictedRaceTime (s)
0               0          Oscar Piastri              91.948003
1               1         George Russell              91.948003
2               2           Lando Norris              91.957336
3               3         Max Verstappen              91.957336
4               4         Lewis Hamilton              92.123552
5               5        Charles Leclerc              92.123552
6               6           Isack Hadjar              92.123552
7               7  Andrea Kimi Antonelli              92.123552
8               8           Yuki Tsunoda              93.172026
9               9        Alexander Albon              93.590350
10             10           Esteban Ocon              91.826958
11             11        Nico Hülkenberg              91.826958
12             12        Fernando Alonso              93.498511
13             13         

'Oscar Piastri'